<a href="https://colab.research.google.com/github/Waleed421/DCGAN-image-generator/blob/main/DCGAN_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.1 MB/s eta 0:00:00


Download and Normalize Images: Retrieve the AFHQ dataset and normalize the images to the range [−1, 1] to stabilize GAN training.

In [2]:
from datasets import load_dataset
import torchvision.transforms as transforms

# Load the dataset
dataset = load_dataset("huggan/AFHQ", split="train")

# Define transformation: resize to 64x64 and normalize to [-1, 1]
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Apply transformation
def transform_examples(examples):
    examples['image'] = [transform(image) for image in examples['image']]
    return examples

dataset.set_transform(transform_examples)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


dataset_infos.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/358M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/371M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16130 [00:00<?, ? examples/s]

Create DataLoader: Utilize PyTorch's DataLoader to feed images into the model during training.

In [3]:
from torch.utils.data import DataLoader

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)


Build the GAN Architecture (DCGAN):

Generator Network: Implement the generator using transposed convolutions.

In [4]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, nz):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

Discriminator Network: Implement the discriminator using convolutions, batch normalization, and Leaky ReLU activation.

In [5]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input).view(-1)


Training the GAN:

Initialize Models:

In [6]:
import torch

# Check for CUDA availability and assign device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

nz = 100  # Size of latent vector
netG = Generator(nz).to(device)
netD = Discriminator().to(device)

Define Loss Function and Optimizers:

In [7]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))

Training Loop:

In [8]:
import os
import torch
from torchvision.utils import save_image

def save_generated_images(epoch, images, output_dir='/content/drive/My Drive/GAN_Images', nrow=8):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    # Define the file path
    file_path = os.path.join(output_dir, f'generated_images_epoch_{epoch + 1}.png')
    # Save the image grid
    save_image(images, file_path, nrow=nrow, normalize=True)
    print(f'Saved generated images to {file_path}')


In [9]:
num_epochs = 80
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

for epoch in range(num_epochs):
    print(f'Starting epoch {epoch + 1}/{num_epochs}')
    for i, data in enumerate(dataloader, 0):
        # Update Discriminator
        netD.zero_grad()
        real_images = data['image'].to(device)
        batch_size = real_images.size(0)
        labels = torch.full((batch_size,), 1., device=device)
        output = netD(real_images)
        lossD_real = criterion(output, labels)
        lossD_real.backward()

        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake_images = netG(noise)
        labels.fill_(0.)
        output = netD(fake_images.detach())
        lossD_fake = criterion(output, labels)
        lossD_fake.backward()
        optimizerD.step()

        # Update Generator
        netG.zero_grad()
        labels.fill_(1.)
        output = netD(fake_images)
        lossG = criterion(output, labels)
        lossG.backward()
        optimizerG.step()

    # Save generated images to observe improvements
    if (epoch + 1) % 10 == 0 or epoch == num_epochs - 1:
        with torch.no_grad():
            fake_images = netG(fixed_noise).detach().cpu()
        save_generated_images(epoch, fake_images)


Starting epoch 1/80
Starting epoch 2/80
Starting epoch 3/80
Starting epoch 4/80
Starting epoch 5/80
Starting epoch 6/80
Starting epoch 7/80
Starting epoch 8/80
Starting epoch 9/80
Starting epoch 10/80
Saved generated images to /content/drive/My Drive/GAN_Images/generated_images_epoch_10.png
Starting epoch 11/80
Starting epoch 12/80
Starting epoch 13/80
Starting epoch 14/80
Starting epoch 15/80
Starting epoch 16/80
Starting epoch 17/80
Starting epoch 18/80
Starting epoch 19/80
Starting epoch 20/80
Saved generated images to /content/drive/My Drive/GAN_Images/generated_images_epoch_20.png
Starting epoch 21/80
Starting epoch 22/80
Starting epoch 23/80
Starting epoch 24/80
Starting epoch 25/80
Starting epoch 26/80
Starting epoch 27/80
Starting epoch 28/80
Starting epoch 29/80
Starting epoch 30/80
Saved generated images to /content/drive/My Drive/GAN_Images/generated_images_epoch_30.png
Starting epoch 31/80
Starting epoch 32/80
Starting epoch 33/80
Starting epoch 34/80
Starting epoch 35/80
S

In [10]:
torch.save(netG.state_dict(), "generator.pth")